In [39]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing 
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt


df1 = pd.read_csv('data.train.csv')
df2 = pd.read_csv('data.test.csv')

#csv dosyalarından aldığımız dataların adlarında küçük bir düzenleme

df1.rename( columns={'Unnamed: 0':'Customer', 'Products bought by the customer':'ItemId', 'New products bought by the customer':'Product'}, inplace=True )
df2.rename( columns={'Unnamed: 0':'Customer', 'Products bought by the customers':'ItemId', 'New products bought by the customers':'Product'}, inplace=True )

#Data da verilen ürünler için ItemId kullanıyoruz. Şimdi bir fonksiyonla ItemId sütünunda bulunan tüm ürünleri liste yapalım.

def products(item):
    productList = item.split(";")
    return productList


df1["ItemId"] = df1["ItemId"].apply(products)
df2["ItemId"] = df2["ItemId"].apply(products)

#Liste şeklinde olan ürünler dataframe olarak pek kullanışlı olmayacaktır. Listedeki ürünleri tek tek çıkarıp dataya ekleyelim.
#df1 de boşlukları silersek gereksiz sıralar silinmiş olacaktır.


dfTrain = df1.ItemId.apply(pd.Series) \
    .merge(df1, left_index = True, right_index = True) \
    .drop(["ItemId"], axis = 1) \
    .melt(id_vars = ['Customer', 'Product'], value_name = "ItemId") \
    .drop("variable", axis = 1) \
    .dropna()

dfTest= df2.ItemId.apply(pd.Series) \
    .merge(df2, left_index = True, right_index = True) \
    .drop(["ItemId"], axis = 1) \
    .melt(id_vars = ['Customer', 'Product'], value_name = "ItemId") \
    .drop("variable", axis = 1)

In [44]:


X_train = dfTrain.iloc[:, 2].values
X_train = X_train.astype('U')
X_test = dfTest.iloc[:, 2].values
X_test = X_test.astype('U') 

y_train = dfTrain.iloc[:, 1].values
y_train = y_train.astype('U') 
y_test = dfTest.iloc[:, 1].values
y_test = y_test.astype('U') 

X_train = X_train.reshape(1, -1)
y_train = y_train.reshape(1, -1)
X_test = X_test.reshape(1, -1)
y_test = y_test.reshape(1, -1)

X_train

array([['Cips', 'Patates', 'Kırmızı şarap', ..., 'Kalem', 'Nar ekşisi',
        'Patates']], dtype='<U24')

In [43]:
training_accuracy = []
test_accuracy = []

# We define a range of 1 to 10 (included) neighbors that will be tested
neighbors_settings = range(1,11)

# We loop the KNN model through the range of possible neighbors to evaluate which one would be 
# appropriate for this analysis

for n_neighbors in neighbors_settings:
    
    # creating the KNN classifier
    clf = KNeighborsClassifier(n_neighbors=n_neighbors)
    # fitting the model
    clf.fit(X_train, y_train)
    #recording the accuracy of the training set
    training_accuracy.append(clf.score(X_train, y_train))
    #recording the accuracy of the test set
    test_accuracy.append(clf.score(X_test, y_test))
    
# Data Visualization - Evaluating the accuracy of both the training and the testing sets against 
# n_neighbors
    
plt.plot(neighbors_settings, training_accuracy, label='Accuracy of the Training Set')
plt.plot(neighbors_settings, test_accuracy, label='Accuracy of the Test Set')
plt.ylabel('Accuracy')
plt.xlabel('Number of Neighbors')
plt.legend()

ValueError: could not convert string to float: 'Cips'